# GHZ state
$\newcommand{\bra}[1]{\left\langle{#1}\right|}$
$\newcommand{\ket}[1]{\left|{#1}\right\rangle}$

The purpose of this notebook is to simulate the GHZ experiment
described in the IBM Quantum Experience tutorial in the section
entitled 

>Multiple Qubits, Gates, and Entangled States/GHZ states
    
If you understand our "Bell_and_CHSH_inequalities" notebook,
this notebook uses very similar math.

It uses the following results whose proofs use techniques already covered 
in our "Bell_and_CHSH_inequalities" notebook

$\bra{ b_X} = \bra{ b_Z} H$

$\bra{ b_Y} = \bra{ b_Z}  H S^\dagger$

for $b=0, 1$.

$\bra{\psi} \sigma_A(0) \sigma_B(1) \sigma_C(2)\ket{\psi} = \sum_{b_0 + b_1 + b_2 = 0, 2} Prob(b_0, b_1, b_2) - \sum_{b_0 + b_1 + b_2 = 1, 3} Prob(b_0, b_1, b_2)$

First change your working directory to the qubiter directory in your computer, and add its path to the path environment variable.

In [1]:
import os
import sys
print(os.getcwd())
os.chdir('../')
print(os.getcwd())
sys.path.insert(0,os.getcwd())

/home/jupyter/Notebooks/Quantum/qubiter/jupyter-notebooks
/home/jupyter/Notebooks/Quantum/qubiter


In [2]:
from SEO_writer import *
from SEO_simulator import *
from StateVec import *
import numpy as np

In [3]:
def write_ghz_plus(file_prefix, ghz_only=True, meas=None):
    num_bits = 3
    z_axis = 3
    emb = CktEmbedder(num_bits, num_bits)
    print('-------------------', file_prefix)
    wr = SEO_writer(file_prefix, emb)
    wr.write_one_bit_gate(0, OneBitGates.had2)
    wr.write_one_bit_gate(1, OneBitGates.had2)
    wr.write_one_bit_gate(2, OneBitGates.sigx)

    control_pos = 0
    target_pos = 2
    trols = Controls.new_knob(num_bits, control_pos, kind=True)
    wr.write_controlled_one_bit_gate(
        target_pos, trols, OneBitGates.sigx)

    control_pos = 1
    target_pos = 2
    trols = Controls.new_knob(num_bits, control_pos, kind=True)
    wr.write_controlled_one_bit_gate(
        target_pos, trols, OneBitGates.sigx)

    wr.write_one_bit_gate(0, OneBitGates.had2)
    wr.write_one_bit_gate(1, OneBitGates.had2)
    wr.write_one_bit_gate(2, OneBitGates.had2)

    if not ghz_only:
        for pos in range(3):
            m = meas[pos]
            if m == 1:
                wr.write_one_bit_gate(pos, OneBitGates.had2)
            elif m == 2:
                wr.write_one_bit_gate(pos,
                    OneBitGates.rot_ax, [np.pi/4, z_axis])  # S^\dagger(pos)
                wr.write_one_bit_gate(pos, OneBitGates.had2)
            else:
                assert False
    wr.close_files()
    pic_file = file_prefix + '_' + str(num_bits) + '_ZLpic.txt'
    with open(pic_file) as f:
        print(f.read())
    init_st_vec = StateVec.get_standard_basis_st_vec([0, 0, 0])
    sim = SEO_simulator(file_prefix, num_bits, init_st_vec)
    StateVec.describe_st_vec_dict(sim.cur_st_vec_dict, print_st_vec=True, do_pp=True,
                        omit_zero_amps=True, show_probs=True)
    fin_st_vec = sim.cur_st_vec_dict["pure"]
    print('Prob(bit0=i, bit1=j, bit2=k) for i,j,k,=0,1:')
    prob_arr = np.abs(fin_st_vec)**2
    print(prob_arr)
    mean = prob_arr[0, 0, 0]  \
            + prob_arr[0, 1, 1] \
            + prob_arr[1, 0, 1] \
            + prob_arr[1, 1, 0] \
            - prob_arr[1, 1, 1] \
            - prob_arr[0, 0, 1] \
            - prob_arr[1, 0, 0] \
            - prob_arr[0, 1, 0]
    print('mean=', mean)
    return mean


In [4]:
# sigz(0)sigz(1)sigz(2) measurement
file_prefix = 'io_folder/ghz_zzz_meas'
mean_zzz = write_ghz_plus(file_prefix, ghz_only=True)

------------------- io_folder/ghz_zzz_meas
|   |   H   
|   H   |   
X   |   |   
X---+---@   
X---@   |   
|   |   H   
|   H   |   
H   |   |   

*********branch= pure
state vector:
ZL convention (Zero bit Last in state tuple)
(000)ZL (0.707106781187+0j) , prob= 0.5
(111)ZL (-0.707106781187+0j) , prob= 0.5
total probability of state vector (=one if no measurements)= 1.0
dictionary with key=qubit, value=(Prob(0), Prob(1))
{0: (0.49999999999999961, 0.49999999999999961),
 1: (0.49999999999999961, 0.49999999999999961),
 2: (0.49999999999999961, 0.49999999999999961)}
Prob(bit0=i, bit1=j, bit2=k) for i,j,k,=0,1:
[[[  5.00000000e-01   0.00000000e+00]
  [  1.93646401e-37   0.00000000e+00]]

 [[  0.00000000e+00   1.93646401e-37]
  [  0.00000000e+00   5.00000000e-01]]]
mean= -1.93646400979e-37


In [5]:
# sigy(0)sigy(1)sigx(2) measurement
file_prefix = 'io_folder/ghz_yyx_meas'
mean_yyx = write_ghz_plus(file_prefix, ghz_only=False, meas=[2, 2, 1])


------------------- io_folder/ghz_yyx_meas
|   |   H   
|   H   |   
X   |   |   
X---+---@   
X---@   |   
|   |   H   
|   H   |   
H   |   |   
|   |   Rz  
|   |   H   
|   Rz  |   
|   H   |   
H   |   |   

*********branch= pure
state vector:
ZL convention (Zero bit Last in state tuple)
(000)ZL 0.5j , prob= 0.25
(110)ZL 0.5j , prob= 0.25
(101)ZL 0.5j , prob= 0.25
(011)ZL 0.5j , prob= 0.25
total probability of state vector (=one if no measurements)= 1.0
dictionary with key=qubit, value=(Prob(0), Prob(1))
{0: (0.49999999999999933, 0.49999999999999933),
 1: (0.49999999999999933, 0.49999999999999933),
 2: (0.49999999999999933, 0.49999999999999933)}
Prob(bit0=i, bit1=j, bit2=k) for i,j,k,=0,1:
[[[  2.50000000e-01   6.98526388e-33]
  [  6.88162536e-33   2.50000000e-01]]

 [[  6.98526388e-33   2.50000000e-01]
  [  2.50000000e-01   6.88162536e-33]]]
mean= 1.0


In [6]:
# sigy(0)sigx(1)sigy(2) measurement
file_prefix = 'io_folder/ghz_yxy_meas'
mean_yxy = write_ghz_plus(file_prefix, ghz_only=False, meas=[2, 1, 2])

------------------- io_folder/ghz_yxy_meas
|   |   H   
|   H   |   
X   |   |   
X---+---@   
X---@   |   
|   |   H   
|   H   |   
H   |   |   
|   |   Rz  
|   |   H   
|   H   |   
Rz  |   |   
H   |   |   

*********branch= pure
state vector:
ZL convention (Zero bit Last in state tuple)
(000)ZL 0.5j , prob= 0.25
(110)ZL 0.5j , prob= 0.25
(101)ZL 0.5j , prob= 0.25
(011)ZL 0.5j , prob= 0.25
total probability of state vector (=one if no measurements)= 1.0
dictionary with key=qubit, value=(Prob(0), Prob(1))
{0: (0.49999999999999933, 0.49999999999999933),
 1: (0.49999999999999933, 0.49999999999999933),
 2: (0.49999999999999933, 0.49999999999999933)}
Prob(bit0=i, bit1=j, bit2=k) for i,j,k,=0,1:
[[[  2.50000000e-01   7.26573057e-33]
  [  7.26573057e-33   2.50000000e-01]]

 [[  7.26573057e-33   2.50000000e-01]
  [  2.50000000e-01   7.26573057e-33]]]
mean= 1.0


In [7]:
# sigx(0)sigy(1)sigy(2) measurement
file_prefix = 'io_folder/ghz_xyy_meas'
mean_xyy = write_ghz_plus(file_prefix, ghz_only=False, meas=[1, 2, 2])

------------------- io_folder/ghz_xyy_meas
|   |   H   
|   H   |   
X   |   |   
X---+---@   
X---@   |   
|   |   H   
|   H   |   
H   |   |   
|   |   H   
|   Rz  |   
|   H   |   
Rz  |   |   
H   |   |   

*********branch= pure
state vector:
ZL convention (Zero bit Last in state tuple)
(000)ZL 0.5j , prob= 0.25
(110)ZL 0.5j , prob= 0.25
(101)ZL 0.5j , prob= 0.25
(011)ZL 0.5j , prob= 0.25
total probability of state vector (=one if no measurements)= 1.0
dictionary with key=qubit, value=(Prob(0), Prob(1))
{0: (0.49999999999999933, 0.49999999999999933),
 1: (0.49999999999999933, 0.49999999999999933),
 2: (0.49999999999999933, 0.49999999999999933)}
Prob(bit0=i, bit1=j, bit2=k) for i,j,k,=0,1:
[[[  2.50000000e-01   7.26573057e-33]
  [  7.26573057e-33   2.50000000e-01]]

 [[  7.26573057e-33   2.50000000e-01]
  [  2.50000000e-01   7.26573057e-33]]]
mean= 1.0


In [8]:
# sigx(0)sigx(1)sigx(2) measurement
file_prefix = 'io_folder/ghz_xxx_meas'
mean_xxx = write_ghz_plus(file_prefix, ghz_only=False, meas=[1, 1, 1])

------------------- io_folder/ghz_xxx_meas
|   |   H   
|   H   |   
X   |   |   
X---+---@   
X---@   |   
|   |   H   
|   H   |   
H   |   |   
|   |   H   
|   H   |   
H   |   |   

*********branch= pure
state vector:
ZL convention (Zero bit Last in state tuple)
(100)ZL (0.5+0j) , prob= 0.25
(010)ZL (0.5+0j) , prob= 0.25
(001)ZL (0.5+0j) , prob= 0.25
(111)ZL (0.5+0j) , prob= 0.25
total probability of state vector (=one if no measurements)= 1.0
dictionary with key=qubit, value=(Prob(0), Prob(1))
{0: (0.49999999999999933, 0.49999999999999933),
 1: (0.49999999999999933, 0.49999999999999933),
 2: (0.49999999999999933, 0.49999999999999933)}
Prob(bit0=i, bit1=j, bit2=k) for i,j,k,=0,1:
[[[ 0.    0.25]
  [ 0.25  0.  ]]

 [[ 0.25  0.  ]
  [ 0.    0.25]]]
mean= -1.0


Let

$mean\_abc = \bra{\psi} \sigma_A(0) \sigma_B(1)\sigma_C(2)\ket{\psi}$

where

$\ket{\psi} = \frac{1}{\sqrt{2}}(\ket{000} + \ket{111})$

We expect

$mean\_yyx = mean\_yxy = mean\_xyy = 1$

and 

$mean\_xxx=-1$.

This could never happen classically.

In [9]:
print('-----------------------')
print('mean_yyx =', mean_yyx)
print('mean_yxy =', mean_yxy)
print('mean_xyy =', mean_xyy)
print('mean_xxx =', mean_xxx)

-----------------------
mean_yyx = 1.0
mean_yxy = 1.0
mean_xyy = 1.0
mean_xxx = -1.0
